### Этап 1, градиентный бустинг "в лоб"

In [1]:
#импорт необходимых библиотек и методов
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import GradientBoostingClassifier as GBC

data = pd.read_csv('features.csv', index_col='match_id')

# формирование обучающей выборки и целевой переменной
y_train = data['radiant_win']
X_train = data.drop(['duration','tower_status_radiant','tower_status_dire',
                    'barracks_status_radiant','barracks_status_dire','radiant_win'],axis=1)
l = len(X_train)

1.&nbsp;Какие признаки имеют пропуски среди своих значений (приведите полный список имен этих признаков)? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?

In [2]:
# объект missings содержит имена признаков, 
# содержащих пропущенные значения

missings = pd.Series(X_train.columns[X_train.count() < l])
print missings

0                first_blood_time
1                first_blood_team
2             first_blood_player1
3             first_blood_player2
4             radiant_bottle_time
5            radiant_courier_time
6     radiant_flying_courier_time
7         radiant_first_ward_time
8                dire_bottle_time
9               dire_courier_time
10       dire_flying_courier_time
11           dire_first_ward_time
dtype: object


Данные признаки отвечают за время наступления того или иного события, ("первая кровь", приобретение предметов), а также за параметры, с ними связанные (команда, совершившая "первую кровь" и соответствующие игроки). Пропуски во всех этих признаках могут означать, что событие не наступило за первые пять минут игры. 

2.&nbsp;Как называется столбец, содержащий целевую переменную?

3.&nbsp;Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Какое качество при этом получилось?

In [3]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import GradientBoostingClassifier as GBC
from time import time
#заполнение пустых значений - подходы:
# 1. заполнить нулями (+ для регрессии)
# 2. заполнить большими/маленькими значениями (+ для деревьев)
# 3. заполнить средними или медианами

X_train = X_train.fillna(0.,axis=1)

#инициализация генератора для кросс-валидации и классификации
gen = KFold(n_splits = 5,shuffle=True)
clf1 = GBC(n_estimators = 30, max_depth=2,warm_start=True)

#обучение алгоритма и вычисление времени выполнения
#t0 = time()
#clf1.fit(X_train,y_train)
#print("Настройка модели заняла %0.3f сек." % (time() - t0))

#кросс-валидация и вычисление времени её выполнения
#t0 = time()
#cvs = cross_val_score(estimator=clf1, X=X_train, y=y_train,
#                      cv=gen, scoring='roc_auc')
#print("Оценка качества заняла %0.3f сек." % (time() - t0))

#print cvs, cvs.mean()

4.&nbsp;Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что можно сделать, чтобы ускорить его обучение при увеличении количества деревьев?

Увеличение числа деревьев может улучшить качество классификации, так как отличительной особенностью градиентного бустинга является высокая обобщающая способность. Но добавления большого количества деревьев увеличит время работы алгоритма. В качестве варианта  ускорения обучения при увеличении количества деревьев, можно попробовать упростить обучающую выборку (отсечь наименее важные признаки, уменьшить их размерность либо брать для обучения только часть объектов из выборки) или  упростить модель обучения (ограничить глубину деревьев, использовать "теплый старт").

### Этап 2, логистическая регрессия

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV as LRCV

#масштабирование признаков
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

1.&nbsp;Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем можно объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [5]:
#кросс-валидация логистической регрессии с подбором параметра С
cs = np.geomspace(10e-5,10e4,10)
clf2 = LRCV(Cs=cs,cv = gen,scoring='roc_auc')
t0 = time()
clf2.fit(X_train_scaled, y_train)
print("Оценка качества заняла %0.3f сек." % (time() - t0))
#определение и вывод результатов кросс-валидации при наилучшем С
best_c_index = np.where(cs == clf2.C_)[0][0]
best_cvs = clf2.scores_[1][:, best_c_index]
print 'Среднее оценки качества: ', best_cvs.mean()

Оценка качества заняла 18.504 сек.
Среднее оценки качества:  0.716356701873


Качество логистической регрессии выше, чем у градиентного бустинга. Это можно объяснить тем, что признаки объектов могут быть линейно зависимы, или пространство объектом является линейно разделимым. Логистическая регрессия работает намного быстрее, чем градиентный бустинг.

In [6]:
#удаление категориальных признаков из выборки
categorial = ['lobby_type','r1_hero', 'r2_hero', 'r3_hero',
             'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero',
             'd3_hero', 'd4_hero', 'd5_hero']
X_train_without_cats = X_train.drop(categorial, axis=1)
del categorial[0]
X_train_scaled = scaler.fit_transform(X_train_without_cats)
#кросс-валидация логистической регрессии с подбором параметра С
clf3 = LRCV(Cs=cs,cv = gen,scoring='roc_auc')
t0 = time()
clf3.fit(X_train_scaled, y_train)
print("Оценка качества заняла %0.3f сек." % (time() - t0))
#определение и вывод результатов кросс-валидации при наилучшем С
best_c_index = np.where(cs == clf3.C_)[0][0]
best_cvs = clf3.scores_[1][:, best_c_index]
print 'Среднее оценки качества: ', best_cvs.mean()
#print clf3.coef_


Оценка качества заняла 19.878 сек.
Среднее оценки качества:  0.716429108309


2.&nbsp;Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем можно объяснить это изменение?

Удаление категориальных признаков практически не повлияло на качество классификации. Это можно объяснить тем, что данные признаки имеют низкие веса, то есть являются малозначимыми при классификации.

3.&nbsp;Сколько различных идентификаторов героев существует в данной игре?

In [7]:
unique =  sorted(np.unique(X_train[categorial].values))
unique_len = len(np.unique(X_train[categorial].values))
N = unique[-1]
print N

112


В данных выборки идентификаторы игроков нумеруются от 1 до 112, из них в выборке присутствуют 108.

4.&nbsp;Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем можно это объяснить?

In [8]:
X_pick = np.zeros((data.shape[0], N))
for i, match_id in enumerate(data.index):
    for p in xrange(5):
        X_pick[i, data.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1


X_train_with_bow = np.hstack((X_train_without_cats, X_pick))
X_train_bow_scaled = scaler.fit_transform(X_train_with_bow)
#кросс-валидация логистической регрессии с подбором параметра С
clf4 = LRCV(Cs=cs,cv = gen,scoring='roc_auc')
t0 = time()
clf4.fit(X_train_bow_scaled, y_train)
print("Оценка качества заняла %0.3f сек." % (time() - t0))
#определение и вывод результатов кросс-валидации при наилучшем С
best_c_index = np.where(cs == clf4.C_)[0][0]
best_cvs = clf4.scores_[1][:, best_c_index]
print 'Среднее оценки качества: ', best_cvs.mean()
#print clf4.coef_

Оценка качества заняла 39.594 сек.
Среднее оценки качества:  0.752019865543


Качество классификации улучшилось после добавления в модель "мешка слов". Это можно объяснить тем, замена категориальных признаков в выборке на их векторное представление позволило точнее настроить и обучить модель.
Таким образом, данный классификатор имеет лучший показатель качества AUC-ROC на кросс-валидации и с помощью данной модели будет производиться предсказание.

5.&nbsp;Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?

In [9]:
#Чтение и обработка признаков тестовой выборки
P_data = pd.read_csv('features_test.csv', index_col='match_id')
P_data = P_data.drop(['lobby_type'],axis=1)
P_data = P_data.fillna(0.,axis=1)
X_pick = np.zeros((P_data.shape[0], N))
for i, match_id in enumerate(P_data.index):
    for p in xrange(5):
        X_pick[i, P_data.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, P_data.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
        
X_test = P_data.drop(categorial, axis=1)
X_test = np.hstack((X_test, X_pick))
X_test = scaler.transform(X_test)

#Предсказание и вывод результатов
result = pd.DataFrame(data = clf4.predict_proba(X_test)[:,1],
                   index = P_data.index, columns=['radiant_win_prediction'])
result.to_csv('result.csv')

print 'Минимальное значение прогноза: ', result['radiant_win_prediction'].min()
print 'Максимальное значение прогноза: ', result['radiant_win_prediction'].max()

Минимальное значение прогноза:  0.00849633654525
Максимальное значение прогноза:  0.996279877533
